In [1327]:
# -*- coding: utf-8 -*-
import os, sys
from os.path import dirname, join
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [1328]:
df = pd.read_csv("../databases/risco_credito.csv")
previsores = df.iloc[:, 0:4].values
classe = df.iloc[:, 4].values

In [1329]:
# ATTRIBUTES
# ----------

for c in df.columns:
    print(
        "{0}\nTotal: {1}\n".format(df[c].value_counts(), 
        df['garantias'].value_counts().values.sum()))

boa             5
desconhecida    5
ruim            4
Name: historia, dtype: int64
Total: 14

baixa    7
alta     7
Name: divida, dtype: int64
Total: 14

nenhuma     11
adequada     3
Name: garantias, dtype: int64
Total: 14

acima_35    7
15_35       4
0_15        3
Name: renda, dtype: int64
Total: 14

alto        6
baixo       5
moderado    3
Name: risco, dtype: int64
Total: 14



In [1330]:
# REGRA HISTORIA DE CRÉDITO - ALGORITHM ONER
col_historia = df.iloc[:, 0]
col_risco = df.iloc[:, 4]
col_merge = pd.DataFrame(col_historia).join(col_risco, lsuffix='_caller', rsuffix='_other')

attr_boa = col_merge.loc[col_merge['historia'] == 'boa']
attr_risco_boa = pd.DataFrame(attr_boa)['risco'].value_counts()
attr_historia_boa = pd.DataFrame(attr_boa)['historia'].value_counts()

attr_desc = col_merge.loc[col_merge['historia'] == 'desconhecida']
attr_risco_desc = pd.DataFrame(attr_desc)['risco'].value_counts()
attr_historia_desc= pd.DataFrame(attr_desc)['historia'].value_counts()

attr_ruim = col_merge.loc[col_merge['historia'] == 'ruim']
attr_risco_ruim = pd.DataFrame(attr_ruim)['risco'].value_counts()
attr_historia_ruim = pd.DataFrame(attr_ruim)['historia'].value_counts()

max_value_risco_boa = attr_risco_boa.values.sum() - attr_risco_boa.values.max()
max_value_risco_desc = attr_risco_desc.values.sum() - attr_risco_desc.values.max()
max_value_risco_ruim = attr_risco_ruim.values.sum() - attr_risco_ruim.values.max()

rules_boa = "se for boa então " + attr_risco_boa.index[0]
rules_desc = "se for desc então " + attr_risco_desc.index[0]
rules_ruim = "se for ruim então " + attr_risco_ruim.index[0]

max_value_historia_boa = attr_historia_boa.values.max()
max_value_historia_desc = attr_historia_desc.values.max()
max_value_historia_ruim = attr_historia_ruim.values.max()

error_boa = max_value_risco_boa / max_value_historia_boa
error_desc = max_value_risco_desc / max_value_historia_desc
erro_ruim = max_value_risco_ruim / max_value_historia_ruim

print(
    "Boa: {0} Regras: {1} Erro: {2}/{3}".format(
        max_value_historia_boa, rules_boa,
        max_value_risco_boa, max_value_historia_boa
    )
)
print(
    "Desconhecia: {0} Regras: {1} Erro: {2}/{3}".format(
        max_value_historia_desc, rules_desc,
        max_value_risco_desc, max_value_historia_desc
    )
)
print(
    "Ruim: {0} Regras: {1} Erro: {2}/{3}".format(
        max_value_historia_ruim, rules_ruim,
        max_value_risco_ruim, max_value_historia_ruim
    )
)
print(
    "Total: {0}/{1}".format((max_value_risco_boa + max_value_risco_desc + max_value_risco_ruim), col_historia.value_counts().values.sum()))

Boa: 5 Regras: se for boa então baixo Erro: 2/5
Desconhecia: 5 Regras: se for desc então alto Erro: 3/5
Ruim: 4 Regras: se for ruim então alto Erro: 1/4
Total: 6/14


In [1331]:
# REGRA DIVIDA DE CRÉDITO - ALGORITHM ONER
col_divida = df.iloc[:, 1]
col_risco = df.iloc[:, 4]
col_merge = pd.DataFrame(col_divida).join(col_risco, lsuffix='_caller', rsuffix='_other')

attr_baixa = col_merge.loc[col_merge.divida == 'baixa']
attr_alta = col_merge.loc[col_merge.divida == 'alta']

attr_risco_baixa = pd.DataFrame(attr_baixa).risco.value_counts()
attr_risco_alta = pd.DataFrame(attr_alta).risco.value_counts()
attr_divida_alta = pd.DataFrame(attr_alta).divida.value_counts()
attr_divida_baixa = pd.DataFrame(attr_baixa).divida.value_counts()

max_value_risco_alta = attr_risco_alta.values.sum() - attr_risco_alta.values.max()
max_value_risco_baixa = attr_risco_baixa.values.sum() - attr_risco_baixa.values.max()
max_value_divida_alta = attr_divida_alta.values.max()
max_value_divida_baixa = attr_divida_baixa.values.max()

rules_alta = "se for alta então " + attr_risco_alta.index[0]
rules_baixa = "se for baixa então " + attr_risco_baixa.index[0]

error_alta = max_value_risco_alta / max_value_divida_alta
error_baixa = max_value_risco_baixa / max_value_divida_baixa

print(
    "Alta: {0} Regras: {1} Erro: {2}/{3}".format(
        max_value_divida_alta, rules_alta,
        max_value_risco_alta, max_value_divida_alta
    )
)
print(
    "Baixa: {0} Regras: {1} Erro: {2}/{3}".format(
        max_value_divida_baixa, rules_baixa,
        max_value_risco_baixa, max_value_divida_baixa
    )
)
print(
    "Total: {0}/{1}".format((max_value_risco_alta + max_value_risco_baixa), col_divida.value_counts().values.sum()))

Alta: 7 Regras: se for alta então alto Erro: 3/7
Baixa: 7 Regras: se for baixa então baixo Erro: 4/7
Total: 7/14


In [1332]:
# Árvores de decisão não podem ser categoricos
labelencoder = LabelEncoder()
previsores[:, 0] = labelencoder.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder.fit_transform(previsores[:, 1])
previsores[:, 2] = labelencoder.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder.fit_transform(previsores[:, 3])

In [1333]:
decisionTree = DecisionTreeClassifier(criterion="entropy")
"""
  Attributes
  -----------
    feature_importances_ : classificar a importância de cada atributo
  
  Methods
  -------
    export_graphviz : dot_data : string
      representação gráfica da árvore de decisção
"""
decisionTree = decisionTree.fit(previsores, classe)

In [1334]:
# export_graphviz(
#     decisionTree,
#     out_file=os.path.abspath(join(dirname(os.getcwd()), 'python/tree_credit_risk.dot')),
#     feature_names=["historia", "divida", "garantias", "renda"],
#     class_names=["alto", "moderado", "baixo"],
#     filled=True,
#     leaves_parallel=True,
# )

In [1335]:
result = decisionTree.predict([[0, 0, 1, 2], [3, 0, 0, 0]])